# Push Hugging Face dataset to ES

- data from hugging face
- push vector data 

In [1]:
import sys
sys.path.append("../..")
from datasets import load_dataset
from src.service.provider import ProviderService
from src.utils.type_utils import create_langchain_doc

DATA_REPO = "BroDeadlines/TEST.basic_test_tdt_dataset"
SPLIT = "INDEX.medium_index_TDT"

In [2]:
dataset = load_dataset(DATA_REPO, split=SPLIT)
provider = ProviderService()
index = SPLIT.lower()

vec_idx = f"vec-{index}"
text_idx = f"text-{index}"

es = provider.get_elasticsearch_store(vec_idx)
bm = provider.get_elasticsearch_bm25(text_idx)

In [3]:
dataset

Dataset({
    features: ['content', 'url', 'doc_id', 'shards', 'splits'],
    num_rows: 344
})

In [4]:
from time import time
start = time()
a = [212]*455
end = time()
print(end - start)

4.863739013671875e-05


In [5]:
start = time()
for row in dataset:
    meta = {"doc_id": row['doc_id']}
    docs = [create_langchain_doc(content=split, metadata=meta) for split in row['splits']]
    es.add_documents(docs)
    bm.add_documents(docs)

end = time()

In [6]:
(end-start) / 60

5.594338897864024

In [8]:
vec_idx

'vec-index.medium_index_tdt'

In [9]:
text_idx

'text-index.medium_index_tdt'